# Evaluation Analysis of Developed Methodology for Trust Estimation

## Libraries

In [ ]:
import re
import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

from matplotlib.colors import Normalize, ListedColormap
from matplotlib.cm import ScalarMappable
from matplotlib.patches import FancyArrowPatch
from scipy import stats
from typing import Tuple

# get the current script's directory
current_directory = os.path.dirname(os.path.abspath(__file__)) if "__file__" in locals() else os.getcwd()
# get the parent directory
parent_directory = os.path.dirname(current_directory)
# add the parent directory to the sys.path at the beginning
sys.path.insert(0, parent_directory)

from utils import constants, common

from optimization import functions

from trusts.model_dynamics import TrustDistribution
from trusts.mle_optimization import MLEOptimization

In [ ]:
pd.set_option("display.max_columns",
              None)

## Choose Whether Comparison Model or Ours

In [ ]:
# baseline is a binary performance case (True), ours is a continuous performance case (False)
is_binary_performance = False

## Functions for Plotting

In [ ]:
def sorting_key(filename: str) -> list:
    # define a sorting key function using regular expressions
    
    # extract all numeric parts
    parts = re.findall(r'\d+', filename)
    return [int(part) for part in parts]

In [ ]:
def plot_beta_distribution(axi: plt,
                           distribution_object: TrustDistribution,
                           trust_label: float,
                           experiment_count: int,
                           title_label_name: str,
                           is_save_fig: bool = False) -> None:
    
    mean_value = distribution_object.get_beta_distribution_mean()
    
    label1 = "Human Trust Model Histogram"
    label2 = "Probability Density Function"
    label3 = "Trust Estimation [Mean Value]"
    label4 = "Trust Measurement [Co-Worker]"
    label5 = f"Absolute Error (%) = {100 * abs(mean_value - trust_label):.1f}"
    
    title_label = f"(\u03B1\u2080={distribution_object.initial_alpha:.1f}; \u03B2\u2080={distribution_object.initial_beta:.1f}; \u03B3={distribution_object.gamma:.4f}; \u03B5={distribution_object.epsilon_reward:.4f}; \u03C9\u02E2={distribution_object.w_success:.4f}; \u03C9\u1da0={distribution_object.w_failure:.4f})"
    
    # generate random samples from the beta distribution
    sample_size = 2000
    samples = stats.beta.rvs(distribution_object.alpha,
                             distribution_object.beta,
                             size=sample_size)

    # plot the histogram of the generated samples
    hist1 = axi.hist(samples,
                     bins=30,
                     density=True,
                     alpha=0.5,
                     color="darkorange",
                     label=label1)

    # plot the probability density function (PDF) of the beta distribution
    x = np.linspace(0, 1, 100)
    pdf = stats.beta.pdf(x,
                         distribution_object.alpha,
                         distribution_object.beta)
    plot1 = axi.plot(x,
                     pdf,
                     color="darkred",
                     linestyle="-",
                     linewidth=3,
                     label=label2)

    # plot the mean value as a horizontal line
    line1 = axi.axvline(x=mean_value,
                        color="darkgreen",
                        linestyle="--",
                        linewidth=3,
                        label=label3)
    
    line2 = axi.axvline(x=trust_label,
                        color="darkblue",
                        linestyle="--",
                        linewidth=2,
                        label=label4)
    
    # check the proximity of mean_value and trust_label
    if mean_value - trust_label < 0.04:

        # if close, adjust the position of the text based on the sign of the difference
        if mean_value > trust_label:
            axi.text(mean_value + 0.008,
                     0.4,
                     f"Estimation: % {100 * mean_value:.1f}",
                     color="darkgreen",
                     fontsize=16,
                     fontweight="bold",
                     rotation=90,
                     ha="left")
        else:
            axi.text(mean_value - 0.004,
                     0.4,
                     f"Estimation: % {100 * mean_value:.1f}",
                     color="darkgreen",
                     fontsize=16,
                     fontweight="bold",
                     rotation=90,
                     ha="right")
    else:
        # adjust the position based on the sign of the difference
        if mean_value > trust_label:
            axi.text(mean_value - 0.006,
                     0.4,
                     f"Estimation: % {100 * mean_value:.1f}",
                     color="darkgreen",
                     fontsize=16,
                     fontweight="bold",
                     rotation=90,
                     ha="right")
        else:
            axi.text(mean_value + 0.03,
                     0.4,
                     f"Estimation: % {100 * mean_value:.1f}",
                     color="darkgreen",
                     fontsize=16,
                     fontweight="bold",
                     rotation=90,
                     ha="left")
    
    axi.text(trust_label - 0.04,
             0.4,
             f"Measurement: % {100 * trust_label:.1f}",
             color="darkblue",
             fontsize=16,
             fontweight="bold",
             rotation=90,
             ha="left")
    
    # create an arrow between mean_value and trust_label
    arrow_params = {
        "arrowstyle": "<|-|>",
        "mutation_scale": 15,
        "color": "black",
        "linewidth": 2,
        "label": label5
    }
    axi.annotate("",
                 xy=(mean_value, 0.07),
                 xytext=(trust_label, 0.07),
                 arrowprops=arrow_params)
    arrow_legend = FancyArrowPatch((0, 0),
                                   (1, 0),
                                   **arrow_params)
    
    if mean_value <= 0.5:
        anchor_posx, anchor_posy = 1.0, 1.0
    else:
        anchor_posx, anchor_posy = 0.65, 1.0
    
    legend = axi.legend(handles=[hist1[2][0], plot1[0], line1, line2, arrow_legend],
                        labels=[label1, label2, label3, label4, label5],
                        loc="upper right",
                        bbox_to_anchor=(anchor_posx, anchor_posy),
                        fancybox=True,
                        shadow=True,
                        fontsize=14,
                        title=f"{title_label_name} Experiment {experiment_count}",
                        title_fontsize="12")
    legend.get_title().set_fontweight("bold")

    # format x-axis ticks as percentages
    axi.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: f'{100 * x:.0f}%'))
    # set x-axis ticks at every 10 percent
    axi.set_xticks(np.arange(0, 1.1, 0.1))
    axi.tick_params(axis="x",
                    labelsize=14)
    
    axi.set_title(title_label,
                  fontsize=14)
    axi.set_xlabel(f"Trust Estimation (%) - Beta Distribution (\u03B1={distribution_object.alpha:.1f}; \u03B2={distribution_object.beta:.1f})",
                   fontsize=15)
    axi.set_ylabel("Probability Density Function (pdf)",
                   fontsize=15)
    axi.grid(True,
            which="both",
            axis="x",
            linestyle="--",
            color="grey",
            alpha=0.7,
            linewidth=0.5)

    if is_save_fig:
        plt.savefig(f"{title_label_name}_experiment_{experiment_count}_trust_distribution.png",
                    dpi=600,
                    bbox_inches="tight",
                    transparent=True)

In [ ]:
def run_through_trajectory(df: pd.DataFrame,
                           column_names: list,
                           distribution_object: TrustDistribution,
                           optimization_object: MLEOptimization) -> Tuple[pd.DataFrame,
                                                                          TrustDistribution]:
    
    # create a column to store estimated trust values; initialize it to zero as a default value
    for column in column_names[:-1]:
        df[column] = 0.0
    
    # create columns for quantile values to represent variance (%5 min and %95 )
    df["TrustEstimationQ5"] = 0.0
    df["TrustEstimationQ95"] = 0.0
    
    reward_array = df["Reward"].to_numpy()
    
    # update distribution parameters (alpha and beta) with current success and failure weights
    for idx, reward in enumerate(reward_array):
        
        # in binary model, the trust prediction distributino only gets updated after the experiment is finished
        if is_binary_performance:
            
            # because the success or failure of the experiment could only be obtained after the scenario is finished,
            # the update of reward function value will be laging from previous scenarios
            if (idx + 1) != constants.TRAJECTORY_SIZE:
                if idx == 0:
                    distribution_object.update_parameters(performance=optimization_object.binary_performance_learning[-1])
            
            else:
                distribution_object.update_parameters(performance=reward_array[idx - 1])
        
        else:
            distribution_object.update_parameters(performance=reward)
        
        # store all important parameters in the dataframe
        df.at[idx, column_names[0]] = distribution_object.alpha
        df.at[idx, column_names[1]] = distribution_object.beta
        df.at[idx, column_names[2]] = distribution_object.initial_alpha
        df.at[idx, column_names[3]] = distribution_object.initial_beta
        df.at[idx, column_names[4]] = distribution_object.w_success
        df.at[idx, column_names[5]] = distribution_object.w_failure
        df.at[idx, column_names[6]] = distribution_object.gamma
        df.at[idx, column_names[7]] = distribution_object.epsilon_reward
        
        # calculate and store estimated trust value only after the reward is obtained
        trust_estimation = distribution_object.get_beta_distribution_mean()
        df.at[idx, column_names[8]] = trust_estimation
        
        # store %5 and %95 quantile values of the distribution
        df.at[idx, "TrustEstimationQ5"] = distribution_object.distribution.ppf(0.05)
        df.at[idx, "TrustEstimationQ95"] = distribution_object.distribution.ppf(0.95)
    
    return df, distribution_object

## Initialization

In [ ]:
configs = functions.setup_config(device="cpu")

## Create Beta Distribution Object to Resembe Trust

In [ ]:
trust_obj = TrustDistribution(initial_alpha=configs.initial_alpha,
                              initial_beta=configs.initial_beta,
                              initial_w_success=configs.initial_w_success,
                              initial_w_failure=configs.initial_w_failure,
                              gamma=configs.gamma,
                              epsilon_reward=configs.epsilon_reward)

## Read Robot Experiment Outputs

In [ ]:
_, results_path = functions.get_directories(parent_directory=parent_directory,
                                            data_folder_name="")
experiments_folder_path = os.path.join(results_path,
                                       "experiments")

In [ ]:
# learning trust behavior stage experiments and results
learning_data_folder_path = os.path.join(experiments_folder_path,
                                         constants.LEARNING_EXPERIMENT_FOLDER)
if os.path.exists(learning_data_folder_path):
    learning_excel_files = os.listdir(learning_data_folder_path)
else:
    print("NOTE: learning experiments are not saved @: ", learning_data_folder_path)

# sort the file list using the custom sorting key
learning_excel_files = sorted(learning_excel_files,
                              key=sorting_key)

In [ ]:
# inference stage experiments and results
inference_data_folder_path = os.path.join(experiments_folder_path,
                                          constants.INFERENCE_EXPERIMENT_FOLDER)
if os.path.exists(inference_data_folder_path):
    inference_excel_files = os.listdir(inference_data_folder_path)
else:
    print("NOTE: Inference experiments are not saved @: ", inference_data_folder_path)

# sort the file list using the custom sorting key
inference_excel_files = sorted(inference_excel_files,
                               key=sorting_key)

## Read Human Trust Measurement Labels

In [ ]:
learning_trust_labels_list = []
for learning_file_name in learning_excel_files:
    trust_match = re.search(r"trust_(\d+)",
                            learning_file_name)
    if trust_match:
        learning_trust_labels_list.append(int(trust_match.group(1)))

In [ ]:
inference_trust_labels_list = []
for inference_file_name in inference_excel_files:
    trust_match = re.search(r"trust_(\d+)",
                            inference_file_name)
    if trust_match:
        inference_trust_labels_list.append(int(trust_match.group(1)))

## Define Constant Parameter List

In [ ]:
# Do Not Change Order
prior_columns = ["TimeStep",
                 "StateNorm1", "StateNorm2", "StateNorm3",
                 "Reward"]
column_names = ["Alpha", "Beta",
                "Alpha0", "Beta0",
                "SuccessWeight", "FailureWeight",
                "Gamma", "EpsilonReward",
                "TrustEstimation", "TrustLabel"]

## Run Through Learning Trust Behavior Experiments

In [ ]:
# in learning trust behavior, the following will be updated:
# alpha, beta, alpha_0, beta_0, w_success, w_failure, gamma, eplison_reward
learning_df = pd.DataFrame(columns=["Experiment"] + prior_columns + column_names)

initial_alpha = 0.5
initial_beta = 0.5
w_success = 0.2
w_failure = 0.2
gamma = 0.1
epsilon_reward = 0.1

for idx, learning_excel in enumerate(learning_excel_files):
    
    learn_df = pd.read_excel(os.path.join(learning_data_folder_path,
                                          learning_excel))
    
    # measured trust value of the particular experiment (measured at the end of the experiment)
    trust_label = learning_trust_labels_list[idx]
    true_trust = trust_label / 100 # from percentage to range [0, 1]
    
    # store measured trust
    learn_df[column_names[9]] = true_trust
    
    # save referring index of the experiment
    learn_df["Experiment"] = idx + 1
    
    # collect all dataframes for validation
    if len(learning_df) == 0: learning_df = learn_df
    else: learning_df = pd.concat([learning_df, learn_df], axis=0, ignore_index=True)
    del learn_df
    
    # optimization object for maximum likelihood estimation (MLE)
    mle_obj = MLEOptimization(learning_experiment_data=learning_df,
                              is_binary_performance=is_binary_performance,
                              seed=constants.RANDOM_SEED)
    
    if is_binary_performance:
        # run optimization (binary)
        trust_obj, initial_alpha, initial_beta, w_success, w_failure = mle_obj.fit([initial_alpha,
                                                                                    initial_beta,
                                                                                    w_success,
                                                                                    w_failure])
    else:
        # run optimization (proposed)
        trust_obj, initial_alpha, initial_beta, w_success, w_failure, gamma, epsilon_reward = mle_obj.fit([initial_alpha,
                                                                                                           initial_beta,
                                                                                                           w_success,
                                                                                                           w_failure,
                                                                                                           gamma,
                                                                                                           epsilon_reward])

    # store important data related to the plotting of the distribution
    learning_df.loc[learning_df["Experiment"] == idx + 1, column_names[0]] = trust_obj.alpha
    learning_df.loc[learning_df["Experiment"] == idx + 1, column_names[1]] = trust_obj.beta
    learning_df.loc[learning_df["Experiment"] == idx + 1, column_names[2]] = trust_obj.initial_alpha
    learning_df.loc[learning_df["Experiment"] == idx + 1, column_names[3]] = trust_obj.initial_beta
    learning_df.loc[learning_df["Experiment"] == idx + 1, column_names[4]] = trust_obj.w_success
    learning_df.loc[learning_df["Experiment"] == idx + 1, column_names[5]] = trust_obj.w_failure
    
    if not is_binary_performance:
        learning_df.loc[learning_df["Experiment"] == idx + 1, column_names[6]] = trust_obj.gamma
        learning_df.loc[learning_df["Experiment"] == idx + 1, column_names[7]] = trust_obj.epsilon_reward
    
    # update reward function value to binary case when binary comparison model is selected
    if is_binary_performance:
        learning_df.loc[learning_df["Experiment"] == idx + 1, prior_columns[4]] = mle_obj.reward_binary
    
    # plot estimation, label, error in the trust distribution
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))
    plot_beta_distribution(axi=ax,
                           distribution_object=trust_obj,
                           trust_label=true_trust,
                           experiment_count=idx + 1,
                           title_label_name="Learning Trust Behavior",
                           is_save_fig=False)
    plt.show()

In [ ]:
learning_df

In [ ]:
if is_binary_performance:
    saving_file_name_1 = "learning_stage_experiment_results_binary.xlsx"
else:
    saving_file_name_1 = "learning_stage_experiment_results.xlsx"

In [ ]:
learning_df.to_excel(saving_file_name_1,
                     index=False)

## Plotting Distributions on One Plot All Together for Trust Learning Stage

In [ ]:
learning_df = pd.read_excel(saving_file_name_1)

alphas = learning_df[column_names[0]].tolist()
betas = learning_df[column_names[1]].tolist()
true_trusts = learning_df[column_names[9]].tolist()
experiment_ids = learning_df["Experiment"].tolist()

last_alpha = None
last_beta = None

fig, ax = plt.subplots(figsize=(10, 6))

unique_experiments = sorted(set(experiment_ids))
colors = plt.cm.inferno(np.linspace(0, 1, len(unique_experiments)))
discrete_cmap = ListedColormap(colors)
norm = Normalize(vmin=min(unique_experiments), vmax=max(unique_experiments))
sm = ScalarMappable(cmap=discrete_cmap, norm=norm)

for idx in range(len(alphas)):
    
    if alphas[idx] != last_alpha:
        
        last_alpha = alphas[idx]
        last_beta = betas[idx]
        
        trust_obj.alpha = last_alpha
        trust_obj.beta = last_beta
        
        mean_value = trust_obj.get_beta_distribution_mean()
        sample_size = 2000
        samples = stats.beta.rvs(trust_obj.alpha, trust_obj.beta, size=sample_size)
        
        color = discrete_cmap(norm(experiment_ids[idx]))
        
        ax.hist(samples, bins=30, density=True, alpha=0.15, color=color)
        
        x = np.linspace(0, 1, 100)
        pdf = stats.beta.pdf(x, trust_obj.alpha, trust_obj.beta)
        ax.plot(x, pdf, color=color, linestyle="-", linewidth=4)

cbar = plt.colorbar(sm, ax=ax, orientation="vertical", fraction=0.03, pad=0.04)
cbar.ax.set_visible(True)
cbar.set_label("Stage 3 Experiments Number [#]", fontsize=22)
cbar.ax.tick_params(labelsize=16)
cbar.set_ticks(unique_experiments)
cbar.set_ticklabels([str(exp) for exp in unique_experiments])
cbar.ax.set_position([0.05, 0.1, 0.03, 0.75])

ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: f'{100 * x:.0f}%'))
ax.set_xticks(np.arange(0, 1.1, 0.1))
ax.tick_params(axis="x", labelsize=18)
ax.tick_params(axis="y", labelsize=16)

# ax.set_title("Learning Trust Behavior Stage Experiment Trust Estimations", fontsize=20)
ax.set_xlabel("Human Trust Estimation (%)", fontsize=24)
ax.set_ylabel("Probability Density Function (pdf)", fontsize=22)
ax.grid(True, which="both", axis="x", linestyle="--", color="grey", alpha=0.7, linewidth=0.8)

plt.tight_layout()

# plt.savefig("learning_stage_experiment_trust_distributions.png", dpi=600, bbox_inches="tight", transparent=True)

plt.show()

## Learning Trust Behavior Stage Trust Measurements vs. Reward Function Values

In [ ]:
learning_stage_experiment_idxs = np.unique(learning_df["Experiment"].to_numpy())

learning_stage_trust_labels_df = learning_df[["TimeStep", "TrustLabel"]]
learning_stage_trust_labels = learning_stage_trust_labels_df[
    learning_stage_trust_labels_df["TimeStep"] == constants.TRAJECTORY_SIZE]["TrustLabel"]

learning_stage_rewards_df = learning_df[["Experiment", "Reward"]]

fig, ax1 = plt.subplots(figsize=(10, 6))

ax2 = ax1.twinx()

bar_width = 0.6
bar_positions = np.unique(learning_df["Experiment"].to_numpy()) - (1.4 * bar_width)

learning_stage_exp_idxs = range(0, len(learning_stage_experiment_idxs))

bars = ax1.bar(bar_positions,
               learning_stage_trust_labels,
               width=bar_width,
               color="#4682B4",
               edgecolor="navy",
               linewidth=1.5,
               label="Human Trust Measurement")

for bar in bars:
    height = bar.get_height() * 100
    
    ax1.text(bar.get_x() + bar.get_width() / 2.0, -0.065,
             f"{height:.0f}%",
             ha="center",
             va="bottom",
             color="navy",
             fontsize=18)

sns.violinplot(x="Experiment",
               y="Reward",
               data=learning_stage_rewards_df, ax=ax2,
               palette="inferno",
               alpha=0.1,
               scale="width",
               width=0.6)

ax1.set_xticks(learning_stage_exp_idxs)
ax1.set_xticklabels(range(1, 11))
ax2.set_xticks(learning_stage_exp_idxs)
ax2.set_xticklabels(range(1, 11))

ax1.set_xlim(-0.5, 10)
ax1.set_ylim(-0.07, 0.95)
ax1.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: f'{100 * x:.0f}%'))
ax2.set_yticks(np.arange(-0.2, 0.65, 0.1))

ax1.set_yticks(np.arange(0, 1.0, 0.1))

ax1.tick_params(axis="x", labelcolor="black", labelsize=20)
ax1.tick_params(axis="y", labelcolor="navy", labelsize=20)
ax2.tick_params(axis="y", labelcolor="black", labelsize=20)

ax1.set_xlabel("Stage 3 Experiments Number [#]", color="black", fontsize=24)
ax1.set_ylabel("Human Trust Measurement (%)", color="navy", fontsize=24)
ax2.set_ylabel("Reward Function Output (rᵩ*)", color="black", fontsize=24)

ax2.grid(True, which="both", axis="y", linestyle="--", color="grey", alpha=0.7, linewidth=0.8, zorder=1)

plt.tight_layout()

# plt.savefig("learning_stage_experiment_trust_reward_scales.png", dpi=600, bbox_inches="tight", transparent=True)

plt.show()

## Run Through Inference Experiments After Tuning Beta Distribution Parameters

In [ ]:
# in learning trust behavior, the following will be constant:
# alpha_0, beta_0, w_success, w_failure, gamma, epsilon_reward
INFERENCE_df = pd.DataFrame(columns=["Experiment"] + prior_columns + column_names)

for idx, inference_excel in enumerate(inference_excel_files):
    inference_df = pd.read_excel(os.path.join(inference_data_folder_path,
                                              inference_excel))
    
    # measured trust value of the particular experiment (measured at the end of the experiment)
    trust_label = inference_trust_labels_list[idx]
    true_trust = trust_label / 100 # from percentage to range [0, 1]
    
    # store measured trust
    inference_df[column_names[9]] = true_trust
    
    # save referring index of the experiment
    inference_df["Experiment"] = idx + 1
    
    # in binary model case, update reward function values
    if is_binary_performance:
        inference_df["Reward"] = mle_obj.binary_performance_inference[idx]
    
    # by running the following, only alpha and beta parameters will be updated
    inference_df, trust_obj = run_through_trajectory(df=inference_df,
                                                     column_names=column_names,
                                                     distribution_object=trust_obj,
                                                     optimization_object=mle_obj)
    
    # collect all dataframes
    if len(inference_df) == 0:
        INFERENCE_df = inference_df
    else:
        INFERENCE_df = pd.concat([INFERENCE_df, inference_df], axis=0, ignore_index=True)
    
    # plot estimation, label, error in the trust distribution
    fig, ax = plt.subplots(1, 1,
                           figsize=(8, 5))
    plot_beta_distribution(axi=ax,
                           distribution_object=trust_obj,
                           trust_label=true_trust,
                           experiment_count=idx + 1,
                           title_label_name="Inference Stage",
                           is_save_fig=False)
    plt.show()

In [ ]:
INFERENCE_df

In [ ]:
if is_binary_performance:
    saving_file_name_2 = "inference_stage_experiment_results_binary.xlsx"
else:
    saving_file_name_2 = "inference_stage_experiment_results.xlsx"

In [ ]:
INFERENCE_df.to_excel(saving_file_name_2,
                      index=False)

## Trust Distribution Shifts in One Scenario (PDF and CDF)

In [ ]:
inference_df = INFERENCE_df[INFERENCE_df["Experiment"] == 2]

alphas = inference_df[column_names[0]].tolist()
betas = inference_df[column_names[1]].tolist()

fig, ax = plt.subplots(figsize=(10, 6))

colors = plt.cm.inferno(np.linspace(0, 1, len(alphas)))
discrete_cmap = ListedColormap(colors)
norm = Normalize(vmin=0, vmax=len(alphas))
sm = ScalarMappable(cmap=discrete_cmap, norm=norm)

for idx in range(len(alphas)):
    
    trust_obj.alpha = alphas[idx]
    trust_obj.beta = betas[idx]

    mean_value = trust_obj.get_beta_distribution_mean()
    sample_size = 2000
    samples = stats.beta.rvs(trust_obj.alpha, trust_obj.beta, size=sample_size)

    color = discrete_cmap(norm(idx + 1))

    ax.hist(samples, bins=30, density=True, alpha=0.2, color=color)

    x = np.linspace(0, 0.5, 50)
    pdf = stats.beta.pdf(x, trust_obj.alpha, trust_obj.beta)
    ax.plot(x, pdf, color=color, linestyle="-", linewidth=4)

cbar = plt.colorbar(sm, ax=ax, orientation="vertical", fraction=0.03, pad=0.04)
cbar.ax.set_visible(True)
cbar.set_label("Timestep (Step Number) [#]", fontsize=24)
cbar.ax.tick_params(labelsize=16)
cbar.set_ticks(range(len(alphas)))
cbar.set_ticklabels([str(exp + 1) for exp in range(len(alphas))])
# cbar.ax.set_position([0.05, 0.2, 0.03, 7])

ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: f'{100 * x:.0f}%'))
ax.set_xticks(np.arange(0, 0.51, 0.1))
ax.tick_params(axis="x", labelsize=18)
ax.tick_params(axis="y", labelsize=16)

ax.set_xlabel("Human Trust Estimation (%)", fontsize=24)
ax.set_ylabel("Probability Density Function (pdf)", fontsize=22)
ax.grid(True, which="both", axis="x", linestyle="--", color="grey", alpha=0.7, linewidth=0.5)

plt.tight_layout()

# plt.savefig("inference_stage_experiment_trust_distribution_one_scenario.png", dpi=600, bbox_inches="tight", transparent=True)

plt.show()

In [ ]:
alphas = inference_df[column_names[0]].tolist()
betas = inference_df[column_names[1]].tolist()

fig, ax = plt.subplots(figsize=(10, 6))

colors = plt.cm.inferno(np.linspace(0, 1, len(alphas)))
discrete_cmap = ListedColormap(colors)
norm = Normalize(vmin=0, vmax=len(alphas))
sm = ScalarMappable(cmap=discrete_cmap, norm=norm)

for idx in range(len(alphas)):
    
    trust_obj.alpha = alphas[idx]
    trust_obj.beta = betas[idx]

    sample_size = 2000
    samples = stats.beta.rvs(trust_obj.alpha, trust_obj.beta, size=sample_size)

    color = discrete_cmap(norm(idx + 1))

#     ax.hist(samples, bins=30, density=True, alpha=0.2, color=color)

    x = np.linspace(0, 0.5, 50)
    cdf = stats.beta.cdf(x, trust_obj.alpha, trust_obj.beta)
    ax.plot(x, cdf, color=color, linestyle="-", linewidth=3)

cbar = plt.colorbar(sm, ax=ax, orientation="vertical", fraction=0.03, pad=0.04)
cbar.ax.set_visible(True)
cbar.set_label("Timestep (Step Number) [#]", fontsize=24)
cbar.ax.tick_params(labelsize=16)
cbar.set_ticks(range(len(alphas)))
cbar.set_ticklabels([str(exp + 1) for exp in range(len(alphas))])

ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, _: f'{100 * x:.0f}%'))
ax.set_xticks(np.arange(0, 0.51, 0.1))
ax.tick_params(axis="x", labelsize=18)
ax.tick_params(axis="y", labelsize=16)

ax.set_xlabel("Human Trust Estimation (%)", fontsize=24)
ax.set_ylabel("Cumulative Distribution Function (CDF)", fontsize=20)
ax.grid(True, which="both", axis="x", linestyle="--", color="grey", alpha=0.7, linewidth=0.5)

plt.tight_layout()

# plt.savefig("inference_stage_experiment_trust_distribution_cdf_scenario.png", dpi=600, bbox_inches="tight", transparent=True)

plt.show()

## Visualize Trajectory of Inference Experiment

In [ ]:
inference_experiment_number = 1

In [ ]:
inference_df = INFERENCE_df[INFERENCE_df["Experiment"] == inference_experiment_number]

state_norms = [constants.MAX_DISTANCE_TO_OBJECT,
               constants.MAX_DISTANCE_TO_TARGET,
               constants.MAX_DISTANCE_TO_GROUND]

normalized_states = inference_df[["StateNorm1",
                                  "StateNorm2",
                                  "StateNorm3"]].to_numpy()

denormalized_states = common.denormalize_state(state_norm=normalized_states,
                                               norm_value_list=state_norms)
# correcting extra length addition on end-effector
denormalized_states[:, 1:3] = denormalized_states[:, 1:3] - constants.ROBOT_BASE_HEIGHT

time_steps = inference_df["TimeStep"].to_numpy()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

plt.plot(time_steps, denormalized_states[:, 0],
         linestyle="--",
         linewidth=5,
         color="darkred",
         label="Distance to Obstacle")
plt.plot(time_steps, denormalized_states[:, 1] + 0.1, # add correction to target (m)
         linestyle=":",
         linewidth=5,
         color="darkgreen",
         label="Distance to Target Position")
plt.plot(time_steps, denormalized_states[:, 2],
         linestyle="-",
         linewidth=5,
         color="darkblue",
         label="Distance to Ground Level")

plt.xticks(np.arange(1, max(time_steps) + 1, 1), fontsize=20)
plt.yticks(np.arange(0.0, 2.4, 0.2), fontsize=20)

ax.grid(True, which="both", axis="y", linestyle="--", color="grey", alpha=0.8, linewidth=0.8)

plt.title(f"Inference Experiment {inference_experiment_number} - Robot Trajectory Execution", fontsize=25)
plt.xlabel("Timestep (Step Number) [#]", fontsize=26)
plt.ylabel("Robot State Vector [meters]", fontsize=26)

plt.legend(fancybox=True, shadow=True, loc="upper right", fontsize=22)

plt.tight_layout()

# plt.savefig(f"inference_stage_experiment_{inference_experiment_number}_robot_state.png",
#             dpi=600, bbox_inches="tight", transparent=True)

plt.show()